# BC 3
## Single tiny network; 
## Input = sub-regions 

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import cifar10
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from scipy import stats
from sklearn.metrics import accuracy_score

In [2]:
NUM_TN = 5
SUB_REGION_SCALE = 14

#dataset specific parameters
NUM_CLASS = 10

In [3]:
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

In [5]:
#Generate cropped train image
X_train_cropped = []
y_train_cropped = []
for i in range (X_train.shape[0]):
  image_cropped = tf.image.random_crop(X_train[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
  image_cropped = np.array(image_cropped)
  X_train_cropped.append(image_cropped)
  y_train_cropped.append(y_train[i])
X_train_cropped = np.array(X_train_cropped)
y_train_cropped = np.array(y_train_cropped)

In [6]:
#Generate cropped test image
X_test_cropped_list = []
y_test_cropped_list = []

for i in range(NUM_TN):
  X_test_cropped = []
  y_test_cropped = []
  for i in range (X_test.shape[0]):
    image_cropped = tf.image.random_crop(X_test[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
    image_cropped = np.array(image_cropped)
    X_test_cropped.append(image_cropped)
    y_test_cropped.append(y_test[i])
  X_test_cropped = np.array(X_test_cropped)
  y_test_cropped = np.array(y_test_cropped)

  X_test_cropped_list.append(X_test_cropped)
  y_test_cropped_list.append(y_test_cropped)

In [7]:
#normalizing dataset
X_train_cropped = X_train_cropped/255

for i in range(NUM_TN):
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [8]:
#transform to vector
X_train_cropped = X_train_cropped.reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

for i in range(NUM_TN):
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

In [9]:
#one-hot encoding
y_train_cropped = to_categorical(y_train_cropped)

for i in range(NUM_TN):
  y_test_cropped_list[i] = to_categorical(y_test_cropped_list[i])

In [10]:
#build ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*3))
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_CLASS, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
#train model
hist = model.fit(X_train_cropped, y_train_cropped, validation_data=(X_test_cropped_list[0], y_test_cropped_list[0]), epochs=5, batch_size=128)

Epoch 1/5
391/391 [==============================] - 2s 4ms/step - loss: 2.1911 - accuracy: 0.1714 - val_loss: 2.0633 - val_accuracy: 0.2263
Epoch 2/5
391/391 [==============================] - 1s 3ms/step - loss: 2.0445 - accuracy: 0.2250 - val_loss: 2.0246 - val_accuracy: 0.2397
Epoch 3/5
391/391 [==============================] - 1s 3ms/step - loss: 2.0110 - accuracy: 0.2453 - val_loss: 1.9785 - val_accuracy: 0.2602
Epoch 4/5
391/391 [==============================] - 1s 3ms/step - loss: 1.9878 - accuracy: 0.2545 - val_loss: 1.9928 - val_accuracy: 0.2553
Epoch 5/5
391/391 [==============================] - 1s 3ms/step - loss: 1.9630 - accuracy: 0.2660 - val_loss: 1.9518 - val_accuracy: 0.2750


## Model Evaluation

In [12]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN):
  y_pred = model.predict(X_test_cropped_list[i])
  y_pred_list.append(y_pred)
    
# Ensemble voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.argmax(np.sum(y_pred_list, axis=0), axis=1)

In [13]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.3197